In [13]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import load_model
import numpy as np
import pickle

encoder_inputs = Input(shape=(None, 2008))
model = load_model('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states
# Define an input sequence and process it.
batch_size = 64  # Batch size for training.
epochs = 2  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

num_encoder_tokens = 2008
num_decoder_tokens = 1981
max_encoder_seq_length = 106
max_decoder_seq_length = 131

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


input_token_index = np.load('input_token_index.npy').item()
target_token_index = np.load('target_token_index.npy').item()
with open('input_texts.pickle', 'rb') as f:
        input_texts = pickle.load(f)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, 0] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')

for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ['Can', 'we', 'make', 'this', 'quick', 'Roxanne', 'Korrine', 'and', 'Andrew', 'Barrett', 'are', 'having', 'an', 'incredibly', 'horrendous', 'public', 'break', 'up', 'on', 'the', 'quad', 'Again']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['Well', 'I', 'thought', 'we', 'd', 'start', 'with', 'pronunciation', 'if', 'that', 's', 'okay', 'with', 'you']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['Not', 'the', 'hacking', 'and', 'gagging', 'and', 'spitting', 'part', 'Please']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['You', 're', 'asking', 'me', 'out', 'That', 's', 'so', 'cute', 'What', 's', '

-
Input sentence: ['do', 'you', 'listen', 'to', 'this', 'crap']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['What', 'crap']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['Me', 'This', 'endless', 'blonde', 'babble', 'I', 'm', 'like', 'boring', 'myself']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['I', 'figured', 'you', 'd', 'get', 'to', 'the', 'good', 'stuff', 'eventually']
Decoded sentence: prisonerearlypurposedrugsprestigedoctordoctordoctorreasonTheirgroupwatchinglongconfidentialmercyPinzonPinzongenedoingreporterreadywhich
-
Input sentence: ['What', 'good', 'stuff']
Decoded sentence: prisonerearlypurposedru

KeyboardInterrupt: 